In [1]:
import pandas as pd

# Read with semicolon delimiter
df_nov = pd.concat([pd.read_csv('../../data/EURUSD/EUR-USD_Minute_2025-11-2'+str(i)+'_UTC.csv', index_col=0) for i in range(4,9)])
df_dic_1 = pd.concat([pd.read_csv('../../data/EURUSD/EUR-USD_Minute_2025-12-0'+str(i)+'_UTC.csv', index_col=0) for i in range(1,5)])
df_dic_2 = pd.concat([pd.read_csv('../../data/EURUSD/EUR-USD_Minute_2025-12-0'+str(i)+'_UTC.csv', index_col=0) for i in range(8,10)])
df_dic_3 = pd.concat([pd.read_csv('../../data/EURUSD/EUR-USD_Minute_2025-12-'+str(i)+'_UTC.csv', index_col=0) for i in range(10,13)])
df_dic_4 = pd.concat([pd.read_csv('../../data/EURUSD/EUR-USD_Minute_2025-12-'+str(i)+'_UTC.csv', index_col=0) for i in range(15,19)])

In [2]:
all_df = pd.concat([df_nov,df_dic_1,df_dic_2,df_dic_3,df_dic_4])

In [3]:
import numpy as np

all_x = all_df["Close"].values.reshape(-1,1)
log_ret = np.log(all_x[1:] / all_x[:-1])
log_ret = log_ret.reshape(-1,1)


In [4]:
log_ret.shape

(12619, 1)

In [5]:

def standarize(exog):
    norm_exog = np.zeros((1,exog.shape[1]-1))
    mean_st = []
    std_st = []
    for i in range(2,exog.shape[1]):
        norm_exog[0,i-2] = (exog[0,i-1] - exog[0,:i].mean())/exog[0,:i].std()
        mean_st.append(exog[0,:i].mean())
        std_st.append(exog[0,:i].std())
    
    return norm_exog.T,mean_st,std_st


norm_exog,mean_st,std_st =  standarize(log_ret.T)
tol = 2.22e-16
norm_exog[np.abs(norm_exog) < tol] = 0.0

In [7]:
df_lags = pd.DataFrame(norm_exog)
df_lags.to_csv("../../data/EURUSD/raw_norm_EURUSD.csv", index=False)

In [6]:
lag_matrix = lambda x,lag: np.array([[x[j+i, 0] for j in range(lag)] for i in range(x.shape[0]-lag)])
lags = 25
df_lag_m = lag_matrix(norm_exog, lags)
target_t_0 = norm_exog[lags:]

In [7]:
df_lag_m.shape

(12593, 25)

In [9]:
df_lag_m.shape

(12593, 25)

In [12]:
df_lags

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1.000000,0.941544,0.237551,-1.276159,-1.384269,-0.726249,-0.895127,0.524358,-0.080506,-0.813659,...,0.791036,0.374120,-0.039277,1.069333,-0.299020,1.338912,1.072917,0.017939,-0.298517,-0.399075
1,0.941544,0.237551,-1.276159,-1.384269,-0.726249,-0.895127,0.524358,-0.080506,-0.813659,-0.758559,...,0.374120,-0.039277,1.069333,-0.299020,1.338912,1.072917,0.017939,-0.298517,-0.399075,0.802510
2,0.237551,-1.276159,-1.384269,-0.726249,-0.895127,0.524358,-0.080506,-0.813659,-0.758559,2.748504,...,-0.039277,1.069333,-0.299020,1.338912,1.072917,0.017939,-0.298517,-0.399075,0.802510,0.346278
3,-1.276159,-1.384269,-0.726249,-0.895127,0.524358,-0.080506,-0.813659,-0.758559,2.748504,0.715411,...,1.069333,-0.299020,1.338912,1.072917,0.017939,-0.298517,-0.399075,0.802510,0.346278,0.227953
4,-1.384269,-0.726249,-0.895127,0.524358,-0.080506,-0.813659,-0.758559,2.748504,0.715411,1.783974,...,-0.299020,1.338912,1.072917,0.017939,-0.298517,-0.399075,0.802510,0.346278,0.227953,2.006766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12588,-0.083145,0.280287,-0.083165,0.062219,0.352978,-0.301278,0.425689,-0.446700,-0.155886,-0.010469,...,0.062345,0.498721,0.280487,-0.083185,-0.228653,0.135048,0.135040,0.280512,0.135010,-0.083230
12589,0.280287,-0.083165,0.062219,0.352978,-0.301278,0.425689,-0.446700,-0.155886,-0.010469,0.062240,...,0.498721,0.280487,-0.083185,-0.228653,0.135048,0.135040,0.280512,0.135010,-0.083230,-0.228724
12590,-0.083165,0.062219,0.352978,-0.301278,0.425689,-0.446700,-0.155886,-0.010469,0.062240,-0.374036,...,0.280487,-0.083185,-0.228653,0.135048,0.135040,0.280512,0.135010,-0.083230,-0.228724,-0.155967
12591,0.062219,0.352978,-0.301278,0.425689,-0.446700,-0.155886,-0.010469,0.062240,-0.374036,-0.519472,...,-0.083185,-0.228653,0.135048,0.135040,0.280512,0.135010,-0.083230,-0.228724,-0.155967,-0.301478


In [11]:
df_lags = pd.DataFrame(df_lag_m)
df_lags.to_csv("../../data/EURUSD/lags_EURUSD.csv", index=False)

df_target = pd.DataFrame(target_t_0)
df_target.to_csv("../../data/EURUSD/target_EURUSD.csv", index=False)

In [13]:
df_target

,0
0,0.802510
1,0.346278
2,0.227953
3,2.006766
4,-0.286737
...,...
12588,-0.228724
12589,-0.155967
12590,-0.301478
12591,0.207855
